# Techniques to handle errors and missing values

## Table of contents:
 - [1 Load column with right data type](#1-Load-column-with-right-data-type)
 - [2 Replace NaN (not a number)](#2-Replace-NaN)
    * [2.1 Replace NaN with a negative constant value](#2-1-Replace-NaN-negative)
    * [2.2 Replace NaN values with a mean value](#2-2-Replace-NaN-mean)
    * [2.3 Replace NaN value with a median value](#2-3-Replace-NaN-median)
 - [3 Remove extra white space](#3-Remove-extra-white-space)
 - [4 Replace a data label](#4-Replace-a-data-label)
 - [5 Inject NaN (not a number)](#5-Inject-NaN)

# 1 Load column with right data type
<a name="1-Load-column-with-right-data-type"></a>

When working with your dataset, it is good practice to check that each column contains the type and format of data you expect. Let’s explore an example.  

Let’s look at the travel agency dataset. They are tracking the popularity of destinations based on temperature by recording whether the customer picks the first, second or third destination. We will use pandas to do this.  

To see the data, run the cells below.  

In [2]:
import pandas as pd # used to import the pandas module into out code with the alias pd 

In [3]:
travel_df =  pd.read_csv('travel_agency.csv', sep=',') # The variable travel_df now holds a DataFrame object 

In [4]:
travel_df # Jupyter will print the result of the last evaluation 

,Date,Temperature_city_1,Temperature_city_2,Temperature_city_3,Which_destination
0,20140910,80.0,32.0,40,1
1,20140911,100.0,50.0,36,2
2,20140912,102.0,55.0,46,1
3,20140913,60.0,20.0,35,3
4,20140914,60.0,NaN,32,3
5,20140915,NaN,57.0,42,2


To print a summary of a DataFrame run the cell below.

In [5]:
travel_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                6 non-null      int64  
 1   Temperature_city_1  5 non-null      float64
 2   Temperature_city_2  5 non-null      float64
 3   Temperature_city_3  6 non-null      int64  
 4   Which_destination   6 non-null      int64  
dtypes: float64(2), int64(3)
memory usage: 368.0 bytes


**Think** 🤔 In this case all the columns have numerical data types (i.e. they contain only numbers). Can you detect a problem? What effect does this have on the date column?

> All the data, even the dates, has been parsed as integers (or, in other cases, as string). 
`20140910` is actually a date `2014-Sept-10`, not the number `20,140,910`.

If this was the case with your own dataset, this could cause errors in your exploration and analysis. The solution is to check that your data matches what you would expect for each column, and explicitly specify the expected data types when reading a DataFrame or changing (casting) the column to the desired type or format . In our example we can use the [`pandas.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) function to achieve that. 

In [6]:
# The next line won’t change the original object, it will return a modified copy instead 
pd.to_datetime(travel_df['Date'], format='%Y%m%d') 

0   2014-09-10
1   2014-09-11
2   2014-09-12
3   2014-09-13
4   2014-09-14
5   2014-09-15
Name: Date, dtype: datetime64[ns]

In [7]:
# We can then assign the new Series object to the Date column in the original DataFrame to override it 
travel_df['Date'] = pd.to_datetime(travel_df['Date'], format='%Y%m%d') 

In [8]:
travel_df # will print the object 

,Date,Temperature_city_1,Temperature_city_2,Temperature_city_3,Which_destination
0,2014-09-10,80.0,32.0,40,1
1,2014-09-11,100.0,50.0,36,2
2,2014-09-12,102.0,55.0,46,1
3,2014-09-13,60.0,20.0,35,3
4,2014-09-14,60.0,NaN,32,3
5,2014-09-15,NaN,57.0,42,2


# 2 Replace NaN (not a number)
<a name="2-Replace-NaN"></a>

Looking at the data from the travel agency you looked at above, you will notice that some of the data has been returned as NaN (not a number). This can happen when there is a value that is either missing, undefined or unrepresentable. You will need to replace them with a more meaningful number. There are a few ways you could do this. 

## 2.1 Replace NaN with a negative constant value
<a name="2-1-Replace-NaN-negative"></a>

Missing values can be replaced by a value that you chose to indicate that these values are distinct from others. This approach can be used to later generate a best-guess value to replace the negative constant value, created by a machine  learning algorithm. The method [`DataFrame.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) can be used to replace all NaN values in a DataFrame. Run the cell below.  

In [9]:
# The next row will fill all missing (NaN) values with -1, it returns a modified copy 

# we can pass True to the parameter inplace to modify the original object, otherwise it won’t affect  
# the original DataFrame 

travel_df.fillna(-1) 

,Date,Temperature_city_1,Temperature_city_2,Temperature_city_3,Which_destination
0,2014-09-10,80.0,32.0,40,1
1,2014-09-11,100.0,50.0,36,2
2,2014-09-12,102.0,55.0,46,1
3,2014-09-13,60.0,20.0,35,3
4,2014-09-14,60.0,-1.0,32,3
5,2014-09-15,-1.0,57.0,42,2


## 2.2 Replace NaN values with a mean value
<a name="2-2-Replace-NaN-mean"></a>

NaN values can also be replaced by the column mean or median value as a way to minimise any risk of inaccuracies caused by a machine learning guess. The example below shows how this could be done for our example dataset using the column mean. More specifically, the `mean()` method calculates the mean of the specified axis for the `DataFrame.mean()`. 

**Note that axis = 1 represents the columns & axis = 0 represents the rows.**

<img src="dataframe.png" width=400>

[reference](http://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition (Links to an external site.))

Run the cells below:

In [10]:
travel_df.mean(axis=0) #this computes the mean of all numeric columns including Which_destination

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


Temperature_city_1    80.4
Temperature_city_2    42.8
Temperature_city_3    38.5
Which_destination      2.0
dtype: float64

In [11]:
travel_df.fillna(travel_df.mean(axis=0))   #this replaces the NaN values found in each column with the mean of each column computed above

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


,Date,Temperature_city_1,Temperature_city_2,Temperature_city_3,Which_destination
0,2014-09-10,80.0,32.0,40,1
1,2014-09-11,100.0,50.0,36,2
2,2014-09-12,102.0,55.0,46,1
3,2014-09-13,60.0,20.0,35,3
4,2014-09-14,60.0,42.8,32,3
5,2014-09-15,80.4,57.0,42,2


## 2.3 Replace NaN values with a median value
<a name="2-3-Replace-NaN-median"></a>

The `.median` method is simliar to `.mean`, but it computes the median value. Run the code below and compare the result to the mean values above.

In [12]:
travel_df.fillna(travel_df.median(axis=0))

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


,Date,Temperature_city_1,Temperature_city_2,Temperature_city_3,Which_destination
0,2014-09-10,80.0,32.0,40,1
1,2014-09-11,100.0,50.0,36,2
2,2014-09-12,102.0,55.0,46,1
3,2014-09-13,60.0,20.0,35,3
4,2014-09-14,60.0,50.0,32,3
5,2014-09-15,80.0,57.0,42,2


**Think** 🤔 Why is it important to differentiate between mean and median? 

> While both mean and median measure central tendency, the median is useful in cases where the mean is not a good representative of the value e.g. in the case of extreme values (outliers) in the data or when the values are on ordinal scale (ranking). For example, consider the list of `[1,2,3,4,100]`, the mean is `22`, while the median is `3` (closer to the small values in the list).

# 3 Remove extra white space
<a name="3-Remove-extra-white-space"></a>

Common errors in data are extra white spaces (e.g. unintended use of the spacebar key on the keyboard). These can be introduced during manual handling such as data entry or copying and pasting of data, or automatic parsing of textual data. In Python, a white space counts as a character. 

In the next example, we’ll use the abalone dataset. This includes data related to the physical characteristics of abalone specimens such as sex, length and weight.  

To begin, run the code cells below

In [13]:
abalone_corrupted = pd.read_csv('abalone_corrupted.csv') 

In [14]:
abalone_corrupted['Sex'].value_counts() 

M     1525
I     1328
F     1307
I       17
Name: Sex, dtype: int64

**Think** 🤔 The code above has called for the output of the number of abalone specimens by their recorded sex (e.g. male, female or infant – infant abalone have not yet developed into male or female), Have another look at the output: can you spot the error? 

> The data shows two values against `I` (e.g. we have split the total number of infant abalone specimens into two parts, where we really just want to know the total number). A common explanation for this type of error is a redundant white space (in this case, in the Sex column in the original dataset). In this column, Python has found two representations of infant: `(I)` and `(I )`. 

How do we fix the error? We can treat the column “Sex” as of type string, and use strip. The method `str.strip()` will delete whitespaces from the beginning and end of a string. Run the code below. 

In [15]:
abalone_corrupted['Sex'] = abalone_corrupted['Sex'].str.strip() 

To see the adjusted output, now rerun the code below.  

In [16]:
abalone_corrupted['Sex'].value_counts() 

M    1525
I    1345
F    1307
Name: Sex, dtype: int64

# 4 Replace a data label
<a name="4-Replace-a-data-label"></a>

Sometimes you might want to change the data value in your output from what was used in the original dataset. For example, the word “juvenile” is more typically used than “infant” when referring to developing abalone specimens. To change `I` to `J` in the output, run the code below. 

In [17]:
abalone_corrupted['Sex'].replace('I', 'J', inplace=True) 
#inplace=True means that the original object is modified, so the renaming will remain

In [18]:
abalone_corrupted['Sex'].value_counts()

M    1525
J    1345
F    1307
Name: Sex, dtype: int64

# 5 Inject NaN (not a number) data
<a name="5-Inject-NaN"></a>

Sometimes you might want to replace a specific data value with a NaN value. In this case, there is a value 222 in the column rings (describing the number of growth rings shown by the abalone specimens). The median number of rings for the specimens is 9, so 222 is an outlier, and it’s likely an error.   

Run the code below to see the result. 

In [19]:
# First we access the columns diameter and rings. We then select all rows where rings equals to 222 
abalone_corrupted[['Diameter', 'Rings']][abalone_corrupted.Rings == 222] 

,Diameter,Rings
232,0.505,222
270,0.525,222
275,0.540,222
351,0.450,222
433,0.420,222
675,0.395,222


In [20]:
# to make sure you understand this syntax, try running the commands separately (either in a new cell or commenting each line) and observe the result 
_df = abalone_corrupted[['Diameter', 'Rings']] # this returns a pandas DataFrame (_df) 
print(f'The result of the first part:\n{_df}') 
_sr = abalone_corrupted.Rings == 222 # this returns a pandas Series (_sr) 
print(f'The result of the second part:\n{_sr}') 
print(f'The result of the third part:\n{_df[_sr]}')
#the f in the text of the print method indicates the string has a format, please refer to: https://docs.python.org/3/reference/lexical_analysis.html#f-strings for more information

The result of the first part:
      Diameter  Rings
0        0.365     15
1        0.265      7
2        0.420      9
3        0.365     10
4        0.255      7
...        ...    ...
4172     0.450     11
4173     0.440     10
4174     0.475      9
4175     0.485     10
4176     0.555     12

[4177 rows x 2 columns]
The result of the second part:
0       False
1       False
2       False
3       False
4       False
        ...  
4172    False
4173    False
4174    False
4175    False
4176    False
Name: Rings, Length: 4177, dtype: bool
The result of the third part:
     Diameter  Rings
232     0.505    222
270     0.525    222
275     0.540    222
351     0.450    222
433     0.420    222
675     0.395    222


In [21]:
import numpy as np #imports numpy library with the name np

In [22]:
# here using loc[] we access all the rows where rings equals to 222 and column Diameter, we assign these values a NaN value 
abalone_corrupted.loc[abalone_corrupted.Rings == 222, 'Diameter'] = np.nan 
abalone_corrupted[['Diameter', 'Rings']][abalone_corrupted.Rings == 222] 

,Diameter,Rings
232,NaN,222
270,NaN,222
275,NaN,222
351,NaN,222
433,NaN,222
675,NaN,222


In the case above, you could then use replace NaN with a more meaningful value (for example, the mean) to resolve the problem. Run the code below to see the result.    

In [23]:
# notice that we use inplace=True here, to modify the original DataFrame object 
abalone_corrupted['Diameter'].fillna(abalone_corrupted['Diameter'].mean(axis=0),inplace=True) 
# display the DataFrame Diameter and Rings columns. This time Diameter has new values after replacing with the mean
abalone_corrupted[['Diameter', 'Rings']][abalone_corrupted.Rings == 222] 

,Diameter,Rings
232,0.407788,222
270,0.407788,222
275,0.407788,222
351,0.407788,222
433,0.407788,222
675,0.407788,222


<div class="warning" style='padding:0.1em; background-color:#e6ffff'>
<span>
<p style='margin:1em;'>
<b>Congratulations!</b></p>
<p style='margin:1em;'>
You’ve now completed this activity, which will assist you in cleaning your data in assignments to come. To revise this activity, return to the Canvas page and read over the content under “What you’ll learn”.
</p>
<p style='margin-bottom:1em; margin-right:1em; text-align:right; font-family:Georgia'>
</p></span>
</div>